In [6]:
# import libraries
import re
import datetime
import time
from urllib.request import urlopen
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.remote.webelement import WebElement
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException


In [7]:

def SP_scraping (urlpage,driver):
    full_data=[]
    number=0
    # get web page
    driver.get(urlpage)
    # execute script to scroll down the page

    delay = 3 # seconds
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
    driver.implicitly_wait(1)
    time.sleep(2)




    # create empty array to store data
    # loop over results
    urlList=[]

    link1 = driver.find_elements_by_xpath("//a[contains(@class,'s-access-detail-page')]")

    print("Number of Link Found: ", len(link1))
    
    
    for c in link1:
        url = c.get_attribute("href")
        print(url)
        urlList.append(url)
        
        title=c.get_attribute("title")
        print(title)
           
        
    print("Links Needed to Process: ", len(urlList))
    
    for url in urlList:
        number += 1
        
        print(number, "Processing: ", '. ', '\u001b[34m', url, "......\u001b[0m")
        t1_start = time.perf_counter()
        driver.get(url)
        time.sleep(3)
        name = driver.find_element_by_xpath("//*[@id='productTitle'] | //*[@id='mas-title']").text.strip()
        print(name)

        if "Alex" in name:
            print('Alexa detected, skiping this one')
            continue
        print (name)
        try:
            description = driver.find_element_by_id('feature-bullets')
            print(description)
            features = description.find_elements_by_css_selector("span.a-list-item")
            print(features)
            product_description = clean(features)
            print(product_description)
        except NoSuchElementException:
            print('Not adding this item')
            continue
        try:
            price =float("".join(re.findall("\d+\.\d+", driver.find_element_by_id('priceblock_ourprice').text)))
        except NoSuchElementException:
            price = 0.0
            pass
        try:
            prereview = driver.find_element_by_id('acrCustomerReviewText').text
            review = int("".join(re.findall("\d+", prereview)))
        except NoSuchElementException:
            review = 0
            pass
        try:
            prerating = driver.find_element_by_id('acrPopover').get_attribute('title')
            print(prerating)
            rating = float("".join(re.findall("\d+\.\d+", prerating)))
        except NoSuchElementException:
            rating = 0
            pass
        try:
            details = driver.find_element_by_id('productDetails_detailBullets_sections1')
            more_details = details.find_elements_by_css_selector('td.a-size-base')
            for b in more_details:
                a = b.text
                if "inches" in a:
                    size1 = a
                    print(size1)
                    size = re.findall(r"[+-]? *(?:\d+(?:\.\d*)?|\.\d+)(?:[eE][+-]?\d+)?", size1)
                    break
                else:
                    size = [0, 0, 0]
            print(size)
            length = float(size[0])
            height = float(size[1])
            depth = float(size[2])
            for d in more_details:
                a = d.text
                print(a)
                if "pounds" in a:
                    weight1 = a
                    weight = float("".join(re.findall(r"[+-]? *(?:\d+(?:\.\d*)?|\.\d+)(?:[eE][+-]?\d+)?", weight1)))
                    print (weight)
                    break
                elif "ounces" in a:
                    weight1 = a
                    weight = (float("".join(re.findall(r"[+-]? *(?:\d+(?:\.\d*)?|\.\d+)(?:[eE][+-]?\d+)?", weight1)))*0.0625)
                    print(weight)
                    break
                else:
                    weight = 0.0
                    print (weight)
                for s in more_details:
                   p = s.text
                   month = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]
                   for b in month:
                       if b in p:
                           listed_date = p
                       else:
                           listed_date = "null"
        except NoSuchElementException:
            length = 0
            height = 0
            depth = 0
            weight = 0
            listed_date = "null"
        link = driver.current_url
        created_date = datetime.datetime.now().isoformat()
        rec ={
                "Name": name,
                "Description": product_description,
                "Price": price,
                "Length": length,
                "Height": height,
                "Depth" : depth,
                "Weight": weight,
                "URL": link,
                "Review": review,
                "Rating": rating,
                "Listed_Date": listed_date,
                "Created_Date": created_date
                #"Image" : image
                }

        

        print(rec)
        full_data.append(rec)
        t1_end = time.perf_counter()
        print('\u001b[31m',t1_end - t1_start, "second used\u001b[0m")
    return full_data

def clean (product_description):
    description=[]
    for desc in product_description:
        ProductDescription = desc.text.strip()
        description.append(ProductDescription)
    enter ="\n"
    return enter.join(description)

def clean_pic (imageID):
    image = str(imageID)
    return (image)


In [8]:
global number
#specify the url
urlpage= "https://www.amazon.com/s/ref=nb_sb_ss_c_1_8?url=search-alias%3Daps&field-keywords=wired+earbuds&sprefix=wired+ea%2Caps%2C173&crid=2450GQYSG85LX"
print(urlpage)

chrome_options = Options()
chrome_options.add_argument('--log-level=3')
driver = webdriver.Chrome("C:\chromedriver_win32\chromedriver.exe")


full_data=SP_scraping(urlpage,driver)

https://www.amazon.com/s/ref=nb_sb_ss_c_1_8?url=search-alias%3Daps&field-keywords=wired+earbuds&sprefix=wired+ea%2Caps%2C173&crid=2450GQYSG85LX
Number of Link Found:  39
https://www.amazon.com/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_electronics_sr_pg1_1?ie=UTF8&adId=A02745543RLHUB75O1Z5X&url=https%3A%2F%2Fwww.amazon.com%2FBetron-Earphones-Headphones-Balanced-Isolating%2Fdp%2FB01M8G5K8V%2Fref%3Dsr_1_1_sspa%2F141-5699935-9644901%3Fs%3Daht%26ie%3DUTF8%26qid%3D1545600408%26sr%3D1-1-spons%26keywords%3Dwired%2Bearbuds%26psc%3D1&qualifier=1545600407&id=3323781290201426&widgetName=sp_atf
Betron ELR50 Earphones Earbuds Headphones, Balanced Bass Driven Sound, Noise Isolating, Stereo for iPhone, iPod, iPad, Samsung and Mp3 Players (Black)
https://www.amazon.com/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_electronics_sr_pg1_2?ie=UTF8&adId=A065041612QBWG7H57BQJ&url=https%3A%2F%2Fwww.amazon.com%2FCancelling-Earphones-RAVEtone-Headphone-Headsets%2Fdp%2FB07G7468KC%2Fref%3Dsr_1_2_sspa%2F1

ROVKING Over Ear in Ear Noise Isolating Sweatproof Sport Headphones Earbuds Earphones w/Remote and Mic Earhook Wired Stereo Workout Earpods for Running Jogging Gym Exercise Cell Phone Ear Buds Black
https://www.amazon.com/Headphones-Earphones-Microphone-Waterproof-Earphone/dp/B07KQ4XNDV/ref=sr_1_29/141-5699935-9644901?s=aht&ie=UTF8&qid=1545600408&sr=1-29&keywords=wired+earbuds
Earbuds Ear Buds Sport Earbuds Running Earbuds in Ear Headphones Wired Earphones with Microphone Mic Stereo and Volume Control Waterproof Wired Earphone Android Mp3 Players Tablet Laptop 3.5mm Audio
https://www.amazon.com/Back-Bay-Sweatproof-Headphones-Microphone/dp/B0757WVSN8/ref=sr_1_30/141-5699935-9644901?s=aht&ie=UTF8&qid=1545600408&sr=1-30&keywords=wired+earbuds
Back Bay 2-in-1 Wireless & Wired Bluetooth Earbuds. Sweatproof Wireless Stereo Headphones with Microphone, 6 Earphone Tips, AUX Cable and Carrying Bag
https://www.amazon.com/gp/slredirect/picassoRedirect.html/ref=pa_sp_btf_electronics_sr_pg1_1?ie=UTF

Active Noise Cancelling Earphones, RAVEtone Wired HD Stereo In Ear Headphone ANC Strong Bass Sports Earbuds with Built In MIC In-line Control Cable Headsets 12 Hours Playtime
Active Noise Cancelling Earphones, RAVEtone Wired HD Stereo In Ear Headphone ANC Strong Bass Sports Earbuds with Built In MIC In-line Control Cable Headsets 12 Hours Playtime
<selenium.webdriver.remote.webelement.WebElement (session="cad3718ad7f04b1875cf09c35461d868", element="0.47468891044774786-2")>
[<selenium.webdriver.remote.webelement.WebElement (session="cad3718ad7f04b1875cf09c35461d868", element="0.47468891044774786-3")>, <selenium.webdriver.remote.webelement.WebElement (session="cad3718ad7f04b1875cf09c35461d868", element="0.47468891044774786-4")>, <selenium.webdriver.remote.webelement.WebElement (session="cad3718ad7f04b1875cf09c35461d868", element="0.47468891044774786-5")>, <selenium.webdriver.remote.webelement.WebElement (session="cad3718ad7f04b1875cf09c35461d868", element="0.47468891044774786-6")>, <sele

Panasonic ErgoFit In-Ear Earbud Headphones RP-HJE120-K (Black) Dynamic Crystal Clear Sound, Ergonomic Comfort-Fit
Panasonic ErgoFit In-Ear Earbud Headphones RP-HJE120-K (Black) Dynamic Crystal Clear Sound, Ergonomic Comfort-Fit
<selenium.webdriver.remote.webelement.WebElement (session="cad3718ad7f04b1875cf09c35461d868", element="0.09792065923576243-2")>
[<selenium.webdriver.remote.webelement.WebElement (session="cad3718ad7f04b1875cf09c35461d868", element="0.09792065923576243-3")>, <selenium.webdriver.remote.webelement.WebElement (session="cad3718ad7f04b1875cf09c35461d868", element="0.09792065923576243-4")>, <selenium.webdriver.remote.webelement.WebElement (session="cad3718ad7f04b1875cf09c35461d868", element="0.09792065923576243-5")>, <selenium.webdriver.remote.webelement.WebElement (session="cad3718ad7f04b1875cf09c35461d868", element="0.09792065923576243-6")>, <selenium.webdriver.remote.webelement.WebElement (session="cad3718ad7f04b1875cf09c35461d868", element="0.09792065923576243-7")>

1.92 ounces
0.12
{'Name': 'Earbuds Headphones Stereo in-Ear Earphones Noise Isolating Wired Earphone with Mic and Volume Control Comfort-Fit for Samsung Galaxy Android Smartphones and Mp3 Players', 'Description': 'ORIGINAL SOUND EXPERIENCES: Excellent stereo headphones with microphone, surround sound effect good. Inside of the speakers have been engineered to maximize sound output and minimize sound loss. Noise elimination. The call is more vivid and clearer. High-quality audio. Suitable for working, running, jogging etc.\nINTELLIGENT LINE CONTROL with MICROPHONE: Earbuds Built-in Microphone With Volume Control System, play / pause music / next track / previous track / answer phone calls / hang up phone calls.\nERGONOMIC DESIGN: Unlike traditional, circular earbuds, the design of the EarPods is defined by the geometry of the ear. Which makes them more comfortable for more people than any other earbud-style headphones, suitable for long-time wearing. And they also can easily pass throug

...

Earbuds Ear Buds Sport Earbuds Running Earbuds in Ear Headphones Wired Earphones with Microphone Mic Stereo and Volume Control Waterproof Wired Earphone Android Mp3 Players Tablet Laptop 3.5mm Audio
Earbuds Ear Buds Sport Earbuds Running Earbuds in Ear Headphones Wired Earphones with Microphone Mic Stereo and Volume Control Waterproof Wired Earphone Android Mp3 Players Tablet Laptop 3.5mm Audio
<selenium.webdriver.remote.webelement.WebElement (session="cad3718ad7f04b1875cf09c35461d868", element="0.36305553304780425-2")>
[<selenium.webdriver.remote.webelement.WebElement (session="cad3718ad7f04b1875cf09c35461d868", element="0.36305553304780425-3")>, <selenium.webdriver.remote.webelement.WebElement (session="cad3718ad7f04b1875cf09c35461d868", element="0.36305553304780425-4")>, <selenium.webdriver.remote.webelement.WebElement (session="cad3718ad7f04b1875cf09c35461d868", element="0.36305553304780425-5")>, <selenium.webdriver.remote.webelement.WebElement (session="cad3718ad7f04b1875cf09c3546

Earbuds Ear Buds Stereo Earphones in-Ear Headphones Earbuds with Microphone Mic and Volume Control Noise Isolating Wired Ear Buds Compatible iPhone Android Phone iPad Tablet Laptop 3.5mm Devices
Earbuds Ear Buds Stereo Earphones in-Ear Headphones Earbuds with Microphone Mic and Volume Control Noise Isolating Wired Ear Buds Compatible iPhone Android Phone iPad Tablet Laptop 3.5mm Devices
<selenium.webdriver.remote.webelement.WebElement (session="cad3718ad7f04b1875cf09c35461d868", element="0.47302152511040596-2")>
[<selenium.webdriver.remote.webelement.WebElement (session="cad3718ad7f04b1875cf09c35461d868", element="0.47302152511040596-3")>, <selenium.webdriver.remote.webelement.WebElement (session="cad3718ad7f04b1875cf09c35461d868", element="0.47302152511040596-4")>, <selenium.webdriver.remote.webelement.WebElement (session="cad3718ad7f04b1875cf09c35461d868", element="0.47302152511040596-5")>, <selenium.webdriver.remote.webelement.WebElement (session="cad3718ad7f04b1875cf09c35461d868", 

Betron B25 Noise Isolating in Ear Canal Headphones Earphones with Pure Sound and Powerful Bass for iPhone, iPad, iPod, Samsung Smartphones and Tablets (Black with Remote)
Betron B25 Noise Isolating in Ear Canal Headphones Earphones with Pure Sound and Powerful Bass for iPhone, iPad, iPod, Samsung Smartphones and Tablets (Black with Remote)
<selenium.webdriver.remote.webelement.WebElement (session="cad3718ad7f04b1875cf09c35461d868", element="0.06501697170711207-2")>
[<selenium.webdriver.remote.webelement.WebElement (session="cad3718ad7f04b1875cf09c35461d868", element="0.06501697170711207-3")>, <selenium.webdriver.remote.webelement.WebElement (session="cad3718ad7f04b1875cf09c35461d868", element="0.06501697170711207-4")>, <selenium.webdriver.remote.webelement.WebElement (session="cad3718ad7f04b1875cf09c35461d868", element="0.06501697170711207-5")>, <selenium.webdriver.remote.webelement.WebElement (session="cad3718ad7f04b1875cf09c35461d868", element="0.06501697170711207-6")>, <selenium.web

3.9 out of 5 stars
1.4 x 2.2 x 6.8 inches
['1.4', ' 2.2', ' 6.8']
1.4 x 2.2 x 6.8 inches
0.0
1.12 ounces
0.07
{'Name': 'Black GUMY in Ear Earbuds with Stay fit Ear Tips. Wired 3.3ft Colored Cord Cable with Headphone Jack. Small, Medium, and Large Ear tip earpieces Included. JVC GUMY HAFX7B', 'Description': '\nPOWERFUL SOUND - Deep base and clean treble for clear sound.\nCOMFORT FIT EAR TIP BUDS - Soft color matched ear buds for perfect fit and sound\nLONG CORD - Long cord 3.3FT with standard headphone jack for easy connect to Android or IPhone with common standard 3.5mm headphone input.\nEXTRA EARTIP EARBUDS INCLUDED - Small, Medium, and Large Eartips are included with every earbuds. Adjust the fit for perfect sound for your ear.\nJVC GUMY - Reliable, Affordable, Great Sounding, and Great Fit\nConnectivity technology : Wired', 'Price': 0.0, 'Length': 1.4, 'Height': 2.2, 'Depth': 6.8, 'Weight': 0.07, 'URL': 'https://www.amazon.com/dp/B073R2RSB2/ref=sxbs_sxwds-deals_3/141-5699935-9644901

5.7 x 1.2 x 2.8 inches
['5.7', ' 1.2', ' 2.8']
5.7 x 1.2 x 2.8 inches
0.0
1.28 ounces
0.08
{'Name': 'ROVKING Wired Sweatproof Earhook in Ear Sport Workout Headphones Noise Isolating Over Ear Earbuds with Microphone for Running Jogging Gym Exercise Earphones, Cell Phone Ear Buds Headset Blue', 'Description': "\nSNUG FIT AND STAY PUT IN YOUR EARS THROUGHOUT WORKOUT - Special designed profile of ROVKING workout earbuds fits snugly into your ear canals. Over the ear design makes the workout headphones keep secure fit. All makes these sport earbuds won't fall out of ears during most active sports. Great sports earbuds for running, jogging, gym, working out, exercising, walking and dancing. They are sweat proof headphones that will still keep staying in place during sweaty workouts.\nLIGHTWEIGHT AND COMFORTABLE FIT - Optimized nozzle angle is designed to ensure comfortbale wearing in the ears. These exercise earbuds are 16g lightweight with soft wire wrapped around the ears. They are comfort

In [9]:
full_data

[{'Name': 'Betron ELR50 Earphones Earbuds Headphones, Balanced Bass Driven Sound, Noise Isolating, Stereo for iPhone, iPod, iPad, Samsung and Mp3 Players (Black)',
  'Description': 'Lightweight in-ear stereo earphones provides natural clear and super enhanced bass sound\nCompatible with all Apple Devices, Android, Smartphone, MP3 player, Laptop, Tablets, other devices with 3.5mm headphone port.\nAll metal housings tuned for accurate music reproduction and amazing clear BASS.\nComfortable In-ear design earphones with soft silicone rubber eartips (S/M/L) which can isolate outside noise with highly qualified sound insulation effect\n1 Year Warranty and 30 days money back guarantee',
  'Price': 10.99,
  'Length': 3.9,
  'Height': 7.3,
  'Depth': 1.0,
  'Weight': 0.21,
  'URL': 'https://www.amazon.com/Betron-Earphones-Headphones-Balanced-Isolating/dp/B01M8G5K8V/ref=sr_1_1_sspa/141-5699935-9644901?s=aht&ie=UTF8&qid=1545600408&sr=1-1-spons&keywords=wired+earbuds&psc=1',
  'Review': 827,
  'Ra

In [10]:
import pandas as pd
#full_df=pd.Da

In [11]:
full_df=pd.DataFrame(full_data)

In [12]:
full_df

,Created_Date,Depth,Description,Height,Length,Listed_Date,Name,Price,Rating,Review,URL,Weight
0,2018-12-23T15:27:04.224105,1.0,Lightweight in-ear stereo earphones provides n...,7.3,3.9,null,"Betron ELR50 Earphones Earbuds Headphones, Bal...",10.99,4.4,827,https://www.amazon.com/Betron-Earphones-Headph...,0.21000
1,2018-12-23T15:27:11.659088,2.4,ACTIVE NOISE CANCELLING: Reduce background sou...,4.1,5.5,null,"Active Noise Cancelling Earphones, RAVEtone Wi...",44.99,4.4,3,https://www.amazon.com/Cancelling-Earphones-RA...,0.38750
2,2018-12-23T15:27:21.007620,0.5,\nNATURAL SOUND QUALITY - Super BASS. 20Hz-20K...,0.5,0.9,null,Earphones Cloudio J1 Noise Cancelling Earbuds ...,19.99,0.0,0,https://www.amazon.com/Cloudio-J1-Cancelling-H...,0.03000
3,2018-12-23T15:27:36.456498,6.8,\nWired In-Ear Headphones: Perfect for exercis...,3.2,3.6,null,Panasonic ErgoFit In-Ear Earbud Headphones RP-...,8.98,4.2,55942,https://www.amazon.com/Panasonic-Headphones-RP...,0.01000
4,2018-12-23T15:27:46.926620,6.8,12mm dome type driver units deliver powerful d...,1.5,2.6,null,Sony MDRXB50AP Extra Bass Earbud Headset (Black),29.99,4.1,3761,https://www.amazon.com/Sony-MDRXB50AP-Extra-Ea...,0.15000
5,2018-12-23T15:27:53.540138,1.4,ORIGINAL SOUND EXPERIENCES: Excellent stereo h...,3.5,3.5,"December 20, 2018",Earbuds Headphones Stereo in-Ear Earphones Noi...,12.99,5.0,162,https://www.amazon.com/Headphones-Earphones-Is...,0.12000
6,2018-12-23T15:28:03.556126,1.0,COMFORTABLE & SNUG FIT PERFECT FOR WORKOUTS: A...,1.4,4.7,null,Avantree Sports Headphones Wired with Micropho...,14.99,4.4,327,https://www.amazon.com/Avantree-Headphones-Mic...,0.06000
7,2018-12-23T15:28:17.838286,8.8,"\nDeep, clear sound enhanced by exclusive TriP...",1.9,4.9,null,Bose SoundSport in-ear headphones for Samsung ...,69.99,4.0,1657,https://www.amazon.com/Bose-SoundSport-headpho...,0.04000
8,2018-12-23T15:28:27.615538,1.2,DUAL DRIVER EARBUDS WITH INCREDIBLE SOUND - Ex...,6.6,9.2,null,Aiwa Prodigy-1 Premium Earphones - in-Ear Head...,29.99,4.3,102,https://www.amazon.com/Aiwa-Prodigy-1-Premium-...,0.15000
9,2018-12-23T15:28:37.388027,1.0,Lightweight in-ear stereo earphones provides n...,7.3,3.9,null,"Betron ELR50 Earphones Earbuds Headphones, Bal...",10.99,4.4,827,https://www.amazon.com/Betron-Earphones-Headph...,0.21000


In [13]:
full_df.to_csv('Scraped_Earbud_Featured_Page.csv',index=False)